In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

In [2]:
model = Network('model')

node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[1].set_number_of_servers(2)

jobclass = np.empty(3, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 2, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 0, node[0], 0)
jobclass[2] = ClosedClass(model, 'Class3', 1, node[0], 0)

node[0].set_service(jobclass[0], Erlang(3, 2))
node[0].set_service(jobclass[1], HyperExp(0.5, 3.0, 10.0))
node[0].set_service(jobclass[2], Exp(1))

node[1].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))
node[1].set_service(jobclass[1], Exp(2))
node[1].set_service(jobclass[2], Exp(3))

P = model.init_routing_matrix()

# P{1,1} = [0.3,0.1; 0.2,0]
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.3)
P.set(jobclass[0], jobclass[0], node[0], node[1], 0.1)
P.set(jobclass[0], jobclass[0], node[1], node[0], 0.2)

# P{1,2} = [0.6,0; 0.8,0]
P.set(jobclass[0], jobclass[1], node[0], node[0], 0.6)
P.set(jobclass[0], jobclass[1], node[1], node[0], 0.8)

# P{2,1} = [0,0; 1,0]
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

# P{2,2} = [0,1; 0,0]
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

# P{3,3} = circul(M)
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)
P.set(jobclass[2], jobclass[2], node[1], node[0], 1.0)

model.link(P)

In [3]:
solver = np.array([], dtype=object)
solver = np.append(solver, CTMC(model))
solver = np.append(solver, JMT(model, seed=23000, samples=5000, verbose=False))
solver = np.append(solver, SSA(model, seed=23000, samples=5000, verbose=False))
solver = np.append(solver, FLD(model))
solver = np.append(solver, MVA(model))
solver = np.append(solver, NC(model, method='exact'))
solver = np.append(solver, MAM(model))
solver = np.append(solver, LINE(model, seed=23000))
solver = np.append(solver, DES(model, seed=23000, samples=5000, verbose=False))

AvgTable = np.empty(len(solver), dtype=object)
AvgChainTable = np.empty(len(solver), dtype=object)
AvgSysTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    try:
        AvgTable[s] = solver[s].avg_table()
        AvgChainTable[s] = solver[s].avg_chain_table()
        AvgSysTable[s] = solver[s].avg_sys_table()
    except Exception as e:
        print(f'Error with {solver[s].get_name()}: {str(e)[:100]}')
        # Set to None for failed solvers
        AvgTable[s] = None
        AvgChainTable[s] = None  
        AvgSysTable[s] = None


SOLVER: CTMC
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.7471 1.7471 0.6667  0.6667 2.6207 2.6207
  Delay   Class3 0.7485 0.7485 1.0000  1.0000 0.7485 0.7485
 Queue1   Class1 0.2529 0.2062 0.1930  0.0965 1.3104 1.3104
 Queue1   Class3 0.2515 0.2203 0.3359  0.3359 0.7485 0.7485
Error with CTMC: too many indices for array: array is 1-dimensional, but 2 were indexed

SOLVER: JMT
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.2247 1.2247 0.6724  0.6724 1.7674 1.7674
  Delay   Class2 0.2188 0.2188 0.2146  0.2146 1.0665 1.0665
  Delay   Class3 0.7455 0.7455 0.9921  0.9921 0.7444 0.7444
 Queue1   Class1 0.0350 0.0190 0.1956  0.1956 0.1814 0.1814
 Queue1   Class2 0.5448 0.2576 0.5233  0.5233 1.0548 1.0548
 Queue1   Class3 0.2545 0.1157 0.3474  0.3474 0.7443 0.7443

SOLVER: SSA
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.1185 1.1185 0.6385  0.6385 1.7518 1.7518
  Delay   Class2 0.2530 0.2530 0